In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [56]:
basedir = './국토교통부_건축물대장_표제부/'
filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv')) & (f.startswith('shared'))]
filenames

['shared_cols_bdinfo_2014.csv',
 'shared_cols_bdinfo_2015.csv',
 'shared_cols_bdinfo_2016.csv',
 'shared_cols_bdinfo_2017.csv',
 'shared_cols_bdinfo_2018.csv',
 'shared_cols_bdinfo_2019.csv',
 'shared_cols_bdinfo_2020.csv']

In [57]:
prac_df = pd.read_csv(basedir + filenames[0])

C:\Users\나\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (14,15,58,59) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [58]:
print(prac_df.shape)
prac_df.head(5)

(644134, 75)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,11305-10019310902,2,집합,3,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1,1.130541e+11,10301.0,0,36.0,0.0,삼일교회,0.0,주건축물,393.00,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),04000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3,0,12.2,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.00,7,80.50,20070821,20070829,20101216.0,2007,3080079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101228
1,11440-2163902,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0,61.0,10.0,NaN,0.0,주건축물,177.62,105.30,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.00,4,46.00,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230
2,11200-2360202,1,일반,2,일반건축물,서울특별시 성동구 송정동 73-566번지,서울특별시성동구 송정14길 17,대한예수교장로회비전교회,11200,11800.0,0.0,73.0,566.0,NaN,NaN,NaN,0,1.120041e+11,11801.0,0,17.0,0.0,NaN,0.0,주건축물,806.00,376.05,46.66,1185.07,797.98,99.00,21.0,철근콘크리트구조,"철근콘크리트,철골조",06000,종교시설,교회,10.0,(철근)콘크리트,"슬라브,경량철골조",0,0,13.2,2,1,0,0,0,0.0,1185.07,0,0.0,0,0.0,0,0.00,0,0.00,NaN,NaN,19781205.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110131
3,11380-2690602,1,일반,2,일반건축물,서울특별시 은평구 응암동 594-87번지,서울특별시은평구 응암로 163-13,NaN,11380,10700.0,0.0,594.0,87.0,NaN,NaN,NaN,0,1.138030e+11,10702.0,0,163.0,13.0,NaN,0.0,주건축물,0.00,0.00,0.00,246.15,0.00,0.00,11.0,벽돌구조,연와조,01000,단독주택,"단독주택,다가구용 (6가구)",10.0,(철근)콘크리트,슬라브,0,6,0.0,2,1,0,0,0,0.0,246.15,0,0.0,0,0.0,0,0.00,0,0.00,19900403,NaN,19911128.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110105
4,11110-10017892502,1,일반,2,일반건축물,서울특별시 종로구 평창동 365-1번지,서울특별시종로구 평창11길 45,NaN,11110,18300.0,0.0,365.0,1.0,NaN,NaN,NaN,0,1.111041e+11,18301.0,0,45.0,0.0,NaN,0.0,주건축물,356.00,104.22,29.28,404.80,194.31,54.58,21.0,철근콘크리트구조,철근콘크리트구조,03000,제1종근린생활시설,제1종근린생활시설,10.0,(철근)콘크리트,(철근)콘크리트,0,0,7.9,2,1,0,0,0,0.0,404.80,0,0.0,0,0.0,1,33.29,2,40.19,20070605,20070616,20080319.0,2007,3000082.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110101


In [59]:
prac_df['대장_구분_코드_명'].value_counts()

일반    514634
집합    129500
Name: 대장_구분_코드_명, dtype: int64

In [61]:
prac_df['대지_위치'].nunique()

574144

In [14]:
for addr in prac_df['대지_위치'].unique():
    addr_df = prac_df[prac_df['대지_위치'] == addr]
    if addr_df.shape[0] > 1:
        print(addr_df.shape)
        display(addr_df)
        break

(8, 75)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
9,11620-165902,1,일반,2,일반건축물,서울특별시 관악구 봉천동 390-3번지,서울특별시관악구 관악로5길 61,서울청룡초등학교,11620,10100.0,0.0,390.0,3.0,NaN,NaN,NaN,0,1.162042e+11,10101.0,0,61.0,0.0,다동,0.0,주건축물,0.0,458.04,0.0,1358.89,1358.89,0.0,21.0,철근콘크리트구조,철근콘크리트구조,10000,교육연구시설,교육연구및복지시설,10.0,(철근)콘크리트,평스라브,0,0,17.55,4,0,0,0,0,0.0,1358.89,0,0.0,0,0.0,0,0.0,2,0.0,NaN,NaN,20040316.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110125
13174,11620-165802,1,일반,2,일반건축물,서울특별시 관악구 봉천동 390-3번지,서울특별시관악구 관악로5길 61,서울청룡초등학교,11620,10100.0,0.0,390.0,3.0,NaN,NaN,NaN,0,1.162042e+11,10101.0,0,61.0,0.0,창고,1.0,부속건축물,0.0,0.00,0.0,36.00,36.00,0.0,11.0,벽돌구조,조적조,10000,교육연구시설,창고,10.0,(철근)콘크리트,평스라브,0,0,0.00,1,0,0,0,0,0.0,36.00,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19850920.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111117
13175,11620-166102,1,일반,2,일반건축물,서울특별시 관악구 봉천동 390-3번지,서울특별시관악구 관악로5길 61,서울청룡초등학교,11620,10100.0,0.0,390.0,3.0,NaN,NaN,NaN,4,1.162042e+11,10101.0,0,61.0,0.0,가동,0.0,주건축물,0.0,0.00,0.0,3307.50,3307.50,0.0,21.0,철근콘크리트구조,철근콘크리트구조,10000,교육연구시설,교육연구및복지시설,10.0,(철근)콘크리트,평스라브,0,0,13.20,4,0,0,0,0,0.0,3307.50,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19850920.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111117
42737,11620-166002,1,일반,2,일반건축물,서울특별시 관악구 봉천동 390-3번지,서울특별시관악구 관악로5길 61,서울청룡초등학교,11620,10100.0,0.0,390.0,3.0,NaN,NaN,NaN,0,1.162042e+11,10101.0,0,61.0,0.0,나동,0.0,주건축물,0.0,0.00,0.0,2385.00,2385.00,0.0,21.0,철근콘크리트구조,철근콘크리트구조,10000,교육연구시설,교육연구및복지시설,10.0,(철근)콘크리트,평스라브,0,0,13.20,4,0,0,0,0,0.0,2385.00,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19850920.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111117
357494,11620-10019294502,1,일반,2,일반건축물,서울특별시 관악구 봉천동 390-3번지,서울특별시관악구 관악로5길 61,서울청룡초등학교 창고,11620,10100.0,0.0,390.0,3.0,NaN,NaN,NaN,0,1.162042e+11,10101.0,0,61.0,0.0,창고,1.0,부속건축물,0.0,51.45,0.0,51.45,51.45,0.0,21.0,철근콘크리트구조,철근콘크리트구조,18000,창고시설,창고,10.0,(철근)콘크리트,평스라브,0,0,0.00,3,0,0,0,0,0.0,51.45,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111125
395430,11620-165602,1,일반,2,일반건축물,서울특별시 관악구 봉천동 390-3번지,서울특별시관악구 관악로5길 61,서울청룡초등학교,11620,10100.0,0.0,390.0,3.0,NaN,NaN,NaN,0,1.162042e+11,10101.0,0,61.0,0.0,창고,1.0,부속건축물,0.0,0.00,0.0,36.00,36.00,0.0,11.0,벽돌구조,조적조,10000,교육연구시설,창고,10.0,(철근)콘크리트,평스라브,0,0,0.00,1,0,0,0,0,0.0,36.00,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19850920.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111117
465559,11620-165502,1,일반,2,일반건축물,서울특별시 관악구 봉천동 390-3번지,서울특별시관악구 관악로5길 61,서울청룡초등학교,11620,10100.0,0.0,390.0,3.0,NaN,NaN,NaN,0,1.162042e+11,10101.0,0,61.0,0.0,창고,1.0,부속건축물,0.0,0.00,0.0,34.20,34.20,0.0,11.0,벽돌구조,조적조,10000,교육연구시설,창고,10.0,(철근)콘크리트,평스라브,0,0,0.00,1,0,0,0,0,0.0,34.20,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19850920.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111117
555952,11620-165702,1,일반,2,일반건축물,서울특별시 관악구 봉천동 390-3번지,서울특별시관악구 관악로5길 61,서울청룡초등학교,11620,10100.0,0.0,390.0,3.0,NaN,NaN,NaN,0,1.162042e+11,10101.0,0,61.0,0.0,창고,1.0,부속건축물,0.0,0.00,0.0,34.20,34.20,0.0,11.0,벽돌구조,조적조,10000,교육연구시설,창고,10.0,(철근)콘크리트,평스라브,0,0,0.00,1,0,0,0,0,0.0,34.20,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19850920.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111117


In [15]:
# 같은 주소라도 여러가지 건축물이 있을 수 있음.

In [16]:
prac_df['주_용도_코드_명'].unique()

array(['제2종근린생활시설', '공동주택', '종교시설', '단독주택', '제1종근린생활시설', '교육연구시설', '창고시설',
       '업무시설', '자동차관련시설', '노유자시설', '문화및집회시설', '숙박시설', '근린생활시설', '공장',
       '위험물저장및처리시설', '묘지관련시설', '교육연구및복지시설', '운동시설', '운수시설', '판매시설',
       '의료시설', '위락시설', '분뇨.쓰레기처리시설', '관광휴게시설', '판매및영업시설', '수련시설',
       '동.식물관련시설', nan, '교정및군사시설', '방송통신시설', '발전시설', '자원순환관련시설', '공공용시설',
       '장례식장'], dtype=object)

In [17]:
prac_df.isna().sum()

관리_건축물대장_PK            0
대장_구분_코드               0
대장_구분_코드_명             0
대장_종류_코드               0
대장_종류_코드_명             0
대지_위치                  0
도로명_대지_위치          49778
건물_명              551669
시군구_코드                 0
법정동_코드                 0
대지_구분_코드               0
번                      0
지                      0
특수지_명             642287
블록                643856
로트                644098
외필지_수                  0
새주소_도로_코드          49775
새주소_법정동_코드         49803
새주소_지상지하_코드            0
새주소_본_번            47493
새주소_부_번            56123
동_명               580752
주_부속_구분_코드            59
주_부속_구분_코드_명          59
대지_면적(㎡)               0
건축_면적(㎡)               0
건폐_율(%)                0
연면적(㎡)                 0
용적_률_산정_연면적(㎡)         0
용적_률(%)                0
구조_코드                 70
구조_코드_명               71
기타_구조                353
주_용도_코드               48
주_용도_코드_명             49
기타_용도                227
지붕_코드                551
지붕_코드_명              578
기타_지붕               2477


In [19]:
prac_df = prac_df.replace('nan', np.nan)

In [20]:
prac_df.isna().sum()

관리_건축물대장_PK            0
대장_구분_코드               0
대장_구분_코드_명             0
대장_종류_코드               0
대장_종류_코드_명             0
대지_위치                  0
도로명_대지_위치          49778
건물_명              551669
시군구_코드                 0
법정동_코드                 0
대지_구분_코드               0
번                      0
지                      0
특수지_명             642287
블록                643856
로트                644098
외필지_수                  0
새주소_도로_코드          49775
새주소_법정동_코드         49803
새주소_지상지하_코드            0
새주소_본_번            47493
새주소_부_번            56123
동_명               580752
주_부속_구분_코드            59
주_부속_구분_코드_명          59
대지_면적(㎡)               0
건축_면적(㎡)               0
건폐_율(%)                0
연면적(㎡)                 0
용적_률_산정_연면적(㎡)         0
용적_률(%)                0
구조_코드                 70
구조_코드_명               71
기타_구조                353
주_용도_코드               48
주_용도_코드_명             49
기타_용도                227
지붕_코드                551
지붕_코드_명              578
기타_지붕               2477


In [21]:
prac_df['주_용도_코드_명'].unique()

array(['제2종근린생활시설', '공동주택', '종교시설', '단독주택', '제1종근린생활시설', '교육연구시설', '창고시설',
       '업무시설', '자동차관련시설', '노유자시설', '문화및집회시설', '숙박시설', '근린생활시설', '공장',
       '위험물저장및처리시설', '묘지관련시설', '교육연구및복지시설', '운동시설', '운수시설', '판매시설',
       '의료시설', '위락시설', '분뇨.쓰레기처리시설', '관광휴게시설', '판매및영업시설', '수련시설',
       '동.식물관련시설', nan, '교정및군사시설', '방송통신시설', '발전시설', '자원순환관련시설', '공공용시설',
       '장례식장'], dtype=object)

In [28]:
prac_df['기타_용도'].nunique()

26627

In [23]:
prac_df['기타_용도'].unique()[:50]

array(['근린생활시설, 공동주택(다세대주택)', '다세대주택', '교회', '단독주택,다가구용 (6가구)',
       '제1종근린생활시설', '근린생활시설 및 다가구주택', '교육연구및복지시설', '교육연구시설', '공동주택(아파트)',
       '주택', '주택 및 사무소 및 영업용', '창고시설', '단독주택', '제1종근린생활시설외1',
       '주택 및 근린생활시설', '업무시설, 근린생활시설', '다가구용단독주택,제1종근린생활시설', '자동차관련시설외 1',
       '다세대주택(8세대)', '다세대주택(10세대)', '다세대 주택', '근린생활시설', '근린생활시설및다가구주택',
       '근린생활시설, 단독주택', '근린생활시설, 주택', '업무시설,근린생활시설', '단독주택 (다가구주택)',
       '지하주차장', '교육연구시설(초등학교)', '제1종근린생활시설,노유자시설', '제2종근린생활시설및다가구주택',
       '다세대주택,제2종근린생활시설', '업무시설,제1종근린생활시설,교육연구시설', '단독주택,다가구용',
       '제2종근린생활시설', '주택, 제2종근린생활시설', '공동주택,다세대', '제1종근린생활시설,제2종근린생활시설',
       '아파트', '창고 및 단독주택', '경비실', '다가구주택', '지하주차장등', '공동주택', '공동주택(다세대)',
       '공동주택,다세대주택', '공동주택(다세대주택)', '공동주택(도시형생활주택)외1', '연립주택', '공동주택,아파트'],
      dtype=object)

In [27]:
prac_df['기타_용도'].unique()[50:100]

array(['제2종근린생활시설(사무소),단독주택', '주택,점포', '제2종근린생활시설외 1', '다가구용단독주택',
       '제1종 및 제2종근린생활시설', '근린생활시설및주택', '영업용', '근린생활시설 및 단독주택',
       '공동주택 (아파트)', '아파트(19세대)', '제2종근린생활시설(수리점,사무실)', '다가구용단독주택(10가구)',
       '연립주택, 공중변소, 관리사무소', '다가구용단독주택(9가구)', '단독주택(다가구주택)',
       '다가구용 단독주택(5가구)', '다세대주택(9세대)', '제1종근린생활시설, 다가구주택',
       '근린생활시설,제2종근린생활시설', '주택 및 사무소', '공동주택(연립주택12세대)', '다세대주택(9가구)',
       '주택및점포', '학교', '자동차관련시설(정비공장)', '근린생활시설,주택', '단독주택(다가구주택 4가구)',
       '대피소', '노유자시설', '집회시설', '주택및여인숙', '근린생활시설 및 다가구용단독주택(4가구)',
       '교육연구시설(교육원)', '다가구용단독주택(5가구)', '전시시설', '다가구단독주택',
       '다가구용 단독주택(3가구)', '종교시설(교회)', '다가구주택, 근린생활시설', '창고', '숙박시설(여관)',
       '다세대주택, 근린생활시설', '다세대주택(7세대)', '다세대주택(6가구)', '근린생활시설 외1',
       '다세대주택,근린생활시설', '자동차관련시설', '사무소', '주택, 근린생활시설(소매점)', '다가구주택(2가구)'],
      dtype=object)

In [29]:
prac_df['기타_용도'].unique()[100:150]

array(['다가구주택(5가구)', '단독주택(다중주택)', '근린생활시설, 다가구주택', '기숙사',
       '다가구주택 및 제1종근린생활시설', '공동주택 및 제2종근린생활시설', '노유자시설(노인복지주택)', '주택, 점포',
       '단독주택(다가구주택), 제1.2종근린생활시설', '펌프실', '교육연구시설(중학교)', '공장시설, 근린생활시설',
       '다세대주택,제1종근린생활시설', '주택, 근린생활시설', '다가구용단독주택(3가구)', '아파트,통신실,재활용창고',
       '종교시설', '단독주택(5가구)', '점포', '교육연구및복지시설(학교)', '다가구용단독주택(8가구)',
       '다가구용 단독주택(6가구)', '근린생활시설 및 주택', '근린생활시실 및 주택', '경비실,경로당,관리사무실',
       '다가구주택(6가구)', '제1종근린생활시설 및 주택', '주택및제2종근린생활시설', '근린생활시설및단독주택',
       '근린생활시설및 다가구용단독주택', '주택및점포및제1.2종근린생활시설', '다가구용단독주택(2가구)', '파출소',
       '관리사무실', '다가구용단독주택, 근린생활시설', '다가구주택(3가구)', '단독주택외2',
       '관리사무소,엠디에프실,방재실,문고,주민공동시설', '소매점,일반음식점,사무소', '주택,근린생활시설',
       '액화석유가스저장시설', '단독주택(다가구)', '공중화장실', '노인복지시설', '다가구용 단독주택',
       '다가구용단독주택(5)', '다가구주택(4가구)', '다가구주택(8가구)', '화장실', '다가구용단독주택(7가구)'],
      dtype=object)

In [ ]:
unique_main_purposes = prac_df['']

In [62]:
unique_etc_purposes = prac_df['기타_용도'].unique()

In [65]:
for purp in unique_etc_purposes:
    if pd.isna(purp) == True:
        pass
    else:
        if '오피' in purp:
            print(purp)

업무시설(오피스텔), 근린생활시설
업무시설(오피스텔)
오피스텔
업무시설(오피스텔),제1,2종근린생활시설
업무시설(오피스텔),공동주택(다세대주택)
업무시설,오피스텔
업무시설(오피스텔224호),제1,2종근린생활시설(10호)
다세대주택(도시형생활주택),제2종근린생활시설(사무소),업무시설(오피스텔)
업무시설(오피스텔),근린생활시설
업무시설(오피스텔 18호),도시형생활주택(29세대),근린생활시설
오피스텔(8실)
아파트,오피스텔,근린생활시설
업무시설(오피스텔)/단독주택
업무시설(오피스텔) 외1
다세대주택(도시형생활주택 12세대), 제2종근린생활시설(사무소),오피스텔
업무시설(오피스텔)/근생
업무시설(오피스텔),판매시설
도시형생활주택,오피스텔
공동주택(다세대주택), 근린생활시설(6호), 오피스텔(2호)
공동주택(도시형생활주택),업무시설(오피스텔),제1종근린생활시설
근린생활시설,업무시설(오피스텔)
업무시설(오피스텔), 제2종근린생활시설
아파트/업무시설(오피스텔)
업무시설(오피스텔-107호) 및 근린생활시설
오피스텔, 주택
제2종근린생활시설(고시원24실)및 업무시설(오피스텔128실)
업무시설(오피스텔-240호세대),공동주택(도시형생활주택 원룸형-115세대)
(오피스텔),다세대주택(도시형생활주택-원룸형),제2종근린생활시설(사무소)
근린생활시설, 공동주택(아파트129세대), 업무시설(오피스텔54실)
공동주택(아파트),업무시설(오피스텔),제1.2종근린생활시설)
근린생활시설,오피스텔,도시형생활주택
업무시설(오피스텔), 교육연구시설, 제1종근린생활시설
업무시설(오피스텔) 및 근린생활시설
제2종근린생활시설(2호) 및 업무시설(오피스텔:60호)
제2종근린생활시설(사무소), 오피스텔, 도시형생활주택
아파트(도시형생활주택),업무시설(오피스텔),제1종근린생활시설(소매점)
오피스텔,도시형생활주택,일반음식점,소매점
업무시설(오피스텔)외 1
업무시설,(오피스텔),제1,2종근린생활시설
다세대주택(도시형생활주택/원룸형 8세대), 업무시설(오피스텔 4호)
공동주택(아파트),오피스텔,근린생활시설
공동주택(

In [66]:
def usage_purpose_living_allocator(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        if ('주택' in x) or ('가구' in x) or ('아파트' in x) or ('주거' in x) or ('세대' in x) or ('오피' in x):
            return '주거용'
        else:
            return '비주거용'

In [67]:
prac_df['주용도_주거용'] = prac_df['주_용도_코드_명'].apply(usage_purpose_living_allocator)

In [68]:
prac_df['기타용도_주거용'] = prac_df['기타_용도'].apply(usage_purpose_living_allocator)

In [69]:
prac_df['주용도_주거용'].unique()

array(['비주거용', '주거용', nan], dtype=object)

In [70]:
prac_df['기타용도_주거용'].unique()

array(['주거용', '비주거용', nan], dtype=object)

In [71]:
prac_df.shape

(644134, 77)

In [43]:
prac_df['주_부속_구분_코드_명'].value_counts()

주건축물     632482
부속건축물     11593
Name: 주_부속_구분_코드_명, dtype: int64

In [72]:
living_df = prac_df[(prac_df['주용도_주거용'] == '주거용') | (prac_df['기타용도_주거용'] == '주거용')]
living_df.shape

(530114, 77)

In [73]:
living_df.head()

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주용도_주거용,기타용도_주거용
0,11305-10019310902,2,집합,3,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1,1.130541e+11,10301.0,0,36.0,0.0,삼일교회,0.0,주건축물,393.00,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),04000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3,0,12.20,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.00,7,80.5,20070821,20070829,20101216.0,2007,3080079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101228,비주거용,주거용
1,11440-2163902,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0,61.0,10.0,NaN,0.0,주건축물,177.62,105.30,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.00,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.00,4,46.0,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,주거용,주거용
3,11380-2690602,1,일반,2,일반건축물,서울특별시 은평구 응암동 594-87번지,서울특별시은평구 응암로 163-13,NaN,11380,10700.0,0.0,594.0,87.0,NaN,NaN,NaN,0,1.138030e+11,10702.0,0,163.0,13.0,NaN,0.0,주건축물,0.00,0.00,0.00,246.15,0.00,0.00,11.0,벽돌구조,연와조,01000,단독주택,"단독주택,다가구용 (6가구)",10.0,(철근)콘크리트,슬라브,0,6,0.00,2,1,0,0,0,0.0,246.15,0,0.0,0,0.0,0,0.00,0,0.0,19900403,NaN,19911128.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110105,주거용,주거용
5,11305-1875702,1,일반,2,일반건축물,서울특별시 강북구 수유동 30-8번지,서울특별시강북구 노해로 47,NaN,11305,10300.0,0.0,30.0,8.0,NaN,NaN,NaN,0,1.130530e+11,10301.0,0,47.0,0.0,NaN,0.0,주건축물,173.10,103.18,59.61,282.46,282.46,163.18,21.0,철근콘크리트구조,철근콘크리트구조,01000,단독주택,근린생활시설 및 다가구주택,10.0,(철근)콘크리트,"(철근)콘크리트,스라브",0,2,11.40,3,0,0,0,0,0.0,282.46,0,0.0,0,0.0,2,42.24,0,0.0,20030407,20030623,20030926.0,2003,NaN,NaN,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110111,주거용,주거용
6,11590-10018266302,2,집합,3,표제부,서울특별시 동작구 상도동 302-28번지,서울특별시동작구 국사봉8길 64-17,두성쉐르빌,11590,10200.0,0.0,302.0,28.0,NaN,NaN,NaN,1,1.159042e+11,10201.0,0,64.0,17.0,NaN,0.0,주건축물,389.48,232.42,59.67,947.74,659.89,169.43,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트,10,0,11.26,4,1,0,0,0,0.0,947.74,0,0.0,0,0.0,8,287.85,2,11.5,20080404,20080405,20080826.0,2008,3190077.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110119,주거용,주거용


In [54]:
not_both_df = living_df[((living_df['주용도_주거용'] == '주거용') & (living_df['기타용도_주거용'] != '주거용'))|
                        (living_df['기타용도_주거용'] == '주거용') & (living_df['주용도_주거용'] != '주거용')
                       ]
not_both_df.shape

(47283, 77)

In [55]:
not_both_df.head()

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주용도_주거용,기타용도_주거용
0,11305-10019310902,2,집합,3,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1,1.130541e+11,10301.0,0,36.0,0.0,삼일교회,0.0,주건축물,393.0,233.81,59.49,761.29,761.29,193.71000,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),04000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3,0,12.20,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.0,7,80.5,20070821,20070829,20101216.0,2007,3080079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101228,비주거용,주거용
13,11410-10018284302,1,일반,2,일반건축물,서울특별시 서대문구 홍제동 278-31번지,서울특별시서대문구 간호대로1길 36,1,11410,11100.0,0.0,278.0,31.0,NaN,NaN,NaN,0,1.141041e+11,11101.0,0,36.0,0.0,.,0.0,주건축물,161.0,96.47,59.92,266.67,266.67,165.63000,21.0,철근콘크리트구조,철근콘크리트구조,04000,제2종근린생활시설,주택 및 사무소 및 영업용,10.0,(철근)콘크리트,평스라브,0,1,8.70,3,0,0,0,0,0.0,266.67,0,0.0,0,0.0,0,0.0,1,0.0,NaN,NaN,19671215.0,2005,3120081.0,건축과,1102.0,증축허가,0,NaN,0.0,0,NaN,0,NaN,0,20100727,비주거용,주거용
39,11620-1885402,1,일반,2,일반건축물,서울특별시 관악구 신림동 1652-20번지,서울특별시관악구 조원로10길 28,NaN,11620,10200.0,0.0,1652.0,20.0,NaN,NaN,NaN,0,1.162042e+11,10201.0,0,28.0,0.0,NaN,0.0,주건축물,158.7,0.00,0.00,0.00,0.00,0.00000,21.0,철근콘크리트구조,"철근콘크리트조, 연와조",03000,제1종근린생활시설,"근린생활시설, 주택",10.0,(철근)콘크리트,슬라브,0,1,12.35,4,1,0,0,0,0.0,0.00,0,0.0,0,0.0,0,0.0,3,34.5,19940328,19940404,19940727.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110512,비주거용,주거용
46,11410-2853302,1,일반,2,일반건축물,서울특별시 서대문구 남가좌동 215-21번지,서울특별시서대문구 거북골로 86-2,NaN,11410,12000.0,0.0,215.0,21.0,NaN,NaN,NaN,0,1.141031e+11,12003.0,0,86.0,2.0,NaN,0.0,주건축물,102.0,75.71,74.23,131.03,124.42,121.98039,11.0,벽돌구조,연와조,04000,제2종근린생활시설,제2종근린생활시설및다가구주택,90.0,기타지붕,기타지붕,0,2,8.00,3,1,0,0,0,0.0,0.00,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19690620.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20090320,비주거용,주거용
58,11320-377502,1,일반,2,일반건축물,서울특별시 도봉구 쌍문동 532-1번지,서울특별시도봉구 삼양로162가길 42-10,NaN,11320,10500.0,0.0,532.0,1.0,NaN,NaN,NaN,0,1.132041e+11,10501.0,0,42.0,10.0,NaN,0.0,주건축물,3242.6,828.00,25.54,828.00,828.00,25.54000,32.0,경량철골구조,조립식(샌드위치단열판넬),18000,창고시설,창고 및 단독주택,90.0,기타지붕,기타지붕,0,1,4.40,1,0,0,0,0,0.0,828.00,0,0.0,0,0.0,0,0.0,6,68.0,19950424,19950508,19950819.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20100417,비주거용,주거용


In [53]:
for addr in living_df['대지_위치'].unique():
    addr_df = living_df[living_df['대지_위치'] == addr]
    
    if addr_df.shape[0] > 1:
        print(addr_df.shape)
        display(addr_df)
        break

(13, 77)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주용도_주거용,기타용도_주거용
11,11740-10017967102,2,집합,3,표제부,서울특별시 강동구 강일동 674번지,서울특별시강동구 아리수로93길 40,강일리버파크,11740,11000.0,0.0,674.0,0.0,NaN,NaN,NaN,0,1.174042e+11,11001.0,0,40.0,0.0,301동,0.0,주건축물,0.0,564.01,0.0,6401.19,6401.19,0.0,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,공동주택(아파트),10.0,(철근)콘크리트,(철근)콘크리트,78,0,45.6,14,0,3,0,0,0.0,6401.19,0,0.0,0,0.0,0,0.0,0,0.0,20041224,20060926,20090311.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110122,주거용,주거용
21423,11740-10017887602,2,집합,3,표제부,서울특별시 강동구 강일동 674번지,서울특별시강동구 아리수로93길 40,강일리버파크,11740,11000.0,0.0,674.0,0.0,NaN,NaN,NaN,0,1.174042e+11,11001.0,0,40.0,0.0,302동,0.0,주건축물,0.0,514.02,0.0,4840.02,4840.02,0.0,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,공동주택(아파트),10.0,(철근)콘크리트,(철근)콘크리트,44,0,37.2,11,0,2,0,0,0.0,4840.02,0,0.0,0,0.0,0,0.0,0,0.0,20041224.0,20060926.0,20090311.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110317,주거용,주거용
40602,11740-10017933202,2,집합,3,표제부,서울특별시 강동구 강일동 674번지,서울특별시강동구 아리수로93길 40,강일리버파크,11740,11000.0,0.0,674.0,0.0,NaN,NaN,NaN,0,1.174042e+11,11001.0,0,40.0,0.0,308동,0.0,주건축물,0.0,825.84,0.0,8267.04,8267.04,0.0,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,공동주택(아파트),10.0,(철근)콘크리트,(철근)콘크리트,128,0,40.0,12,0,3,0,0,0.0,8267.04,0,0.0,0,0.0,0,0.0,0,0.0,20041224,20060926,20090311.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20100918,주거용,주거용
131082,11740-10017892102,2,집합,3,표제부,서울특별시 강동구 강일동 674번지,서울특별시강동구 아리수로93길 40,강일리버파크,11740,11000.0,0.0,674.0,0.0,NaN,NaN,NaN,0,1.174042e+11,11001.0,0,40.0,0.0,303동,0.0,주건축물,0.0,483.31,0.0,4416.17,4416.17,0.0,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,공동주택(아파트),10.0,(철근)콘크리트,(철근)콘크리트,40,0,37.2,11,0,1,0,0,0.0,4416.17,0,0.0,0,0.0,0,0.0,0,0.0,20041224.0,20060926.0,20090311.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20100918,주거용,주거용
141559,11740-10017957602,2,집합,3,표제부,서울특별시 강동구 강일동 674번지,서울특별시강동구 아리수로93길 40,강일리버파크,11740,11000.0,0.0,674.0,0.0,NaN,NaN,NaN,0,1.174042e+11,11001.0,0,40.0,0.0,310동,0.0,주건축물,0.0,668.74,0.0,6671.61,6671.61,0.0,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,공동주택(아파트),10.0,(철근)콘크리트,(철근)콘크리트,94,0,42.8,13,0,3,0,0,0.0,6671.61,0,0.0,0,0.0,0,0.0,0,0.0,20041224.0,20060926,20090311.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20100918,주거용,주거용
232294,11740-10017946102,2,집합,3,표제부,서울특별시 강동구 강일동 674번지,서울특별시강동구 아리수로93길 40,강일리버파크,11740,11000.0,0.0,674.0,0.0,NaN,NaN,NaN,0,1.174042e+11,11001.0,0,40.0,0.0,309동,0.0,주건축물,0.0,846.33,0.0,8354.94,8354.94,0.0,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,공동주택(아파트),10.0,(철근)콘크리트,(철근)콘크리트,114,0,40.0,12,0,3,0,0,0.0,8354.94,0,0.0,0,0.0,0,0.0,0,0.0,20041224.0,20060926,20090311.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20100918,주거용,주거용
289359,11740-10017903702,2,집합,3,표제부,서울특별시 강동구 강일동 674번지,서울특별시강동구 아리수로93길 40,강일리버파크,11740,11000.0,0.0,674.0,0.0,NaN,NaN,NaN,0,1.174042e+11,11001.0,0,40.0,0.0,304동,0.0,주건축물,0.0,483.31,0.0,4867.69,4867.69,0.0,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,공동주택(아파트),10.0,(철근)콘크리트,(철근)콘크리트,44,0,42.3,13,0,1,0,0,0.0,4867.69,0,0.0,0,0.0,0,0.0,0,0.0,20041224.0,20060926.0,20090311.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20140328,주거용,주거용
351687,11740-10017913102,2,집합,3,표제부,서울특별시 강동구 강일동 674번지,서울특별시강동구 아리수로93길 40,강일리버파크,11740,11000.0,0.0,674.0,0.0,NaN,NaN,NaN,0,1.174042e+11,11001.0,0,40.0,0.0,313동,0.0,주건축물,0.0,483.31,0.0,6371.70,6371.70,0.0,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,공동주택(아파트),10.0,(철근)콘크리트,(철근)콘크리트,58,0,48.4,15,0,1,0,0,0.0,6371.70,0,0.0,0,0.0,0,0.0,0,0.0,20041224,20060926,20090311.0,Na

In [76]:
living_df.isna().sum()

관리_건축물대장_PK            0
대장_구분_코드               0
대장_구분_코드_명             0
대장_종류_코드               0
대장_종류_코드_명             0
대지_위치                  0
도로명_대지_위치          39876
건물_명              463224
시군구_코드                 0
법정동_코드                 0
대지_구분_코드               0
번                      0
지                      0
특수지_명             528559
블록                529874
로트                530099
외필지_수                  0
새주소_도로_코드          39875
새주소_법정동_코드         39879
새주소_지상지하_코드            0
새주소_본_번            38334
새주소_부_번            44719
동_명               481422
주_부속_구분_코드             7
주_부속_구분_코드_명           7
대지_면적(㎡)               0
건축_면적(㎡)               0
건폐_율(%)                0
연면적(㎡)                 0
용적_률_산정_연면적(㎡)         0
용적_률(%)                0
구조_코드                 45
구조_코드_명               46
기타_구조                221
주_용도_코드                2
주_용도_코드_명              2
기타_용도                 97
지붕_코드                308
지붕_코드_명              332
기타_지붕               1319


In [78]:
living_df = living_df.dropna(subset=['기타_용도'])
living_df.shape

(530017, 77)

In [79]:
officetel_df = living_df[living_df['기타_용도'].str.contains('오피')]
print(officetel_df.shape)
officetel_df.head()

(1244, 77)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주용도_주거용,기타용도_주거용
1073,11620-1097402,2,집합,3,표제부,서울특별시 관악구 봉천동 948-7번지,서울특별시관악구 봉천로 348,반석오피스텔,11620,10100.0,0.0,948.0,7.0,NaN,NaN,NaN,1,1.162030e+11,10101.0,0,348.0,0.0,NaN,0.0,주건축물,556.25,280.34,50.40,1866.60,1667.87,299.84,21.0,철근콘크리트구조,철근콘크리트조,14000,업무시설,"업무시설(오피스텔), 근린생활시설",10.0,(철근)콘크리트,평슬라브,0,0,20.89,7,1,1,0,0,0.0,1866.60,0,0.0,7,60.0,0,0.00,11,320.15,20010928,20011015,20020926.0,NaN,NaN,NaN,NaN,NaN,61,NaN,0.0,0,NaN,0,NaN,0,20111021,비주거용,주거용
1143,11545-1800202,2,집합,3,표제부,서울특별시 금천구 시흥동 992-67번지,서울특별시금천구 시흥대로57길 19-31,산들파크빌7,11545,10300.0,0.0,992.0,67.0,NaN,NaN,NaN,0,1.154542e+11,10301.0,0,19.0,31.0,산들파크빌7,0.0,주건축물,313.40,149.60,47.73,737.92,737.92,235.46,21.0,철근콘크리트구조,철근콘크리트구조,14000,업무시설,업무시설(오피스텔),10.0,(철근)콘크리트,(철근)콘크리트,0,0,18.50,6,0,1,0,0,0.0,737.92,0,0.0,0,0.0,7,132.48,0,0.00,20031105,20040323,20050126.0,NaN,NaN,NaN,NaN,NaN,17,NaN,0.0,0,NaN,0,NaN,0,20111021,비주거용,주거용
1400,11380-2879402,2,집합,3,표제부,서울특별시 은평구 응암동 118-36번지,서울특별시은평구 응암로29길 18-4,NaN,11380,10700.0,0.0,118.0,36.0,NaN,NaN,NaN,1,1.138041e+11,10701.0,0,18.0,4.0,NaN,0.0,주건축물,289.97,169.00,58.28,987.08,841.75,290.29,21.0,철근콘크리트구조,철근콘크리트구조,14000,업무시설,업무시설(오피스텔),10.0,(철근)콘크리트,평스라브,0,0,17.20,6,0,0,0,0,0.0,987.08,0,0.0,0,0.0,7,145.33,1,0.00,20020620,20030623,20031030.0,2002,3110077.0,건축과,1101.0,신축허가,38,NaN,0.0,0,NaN,0,NaN,0,20111117,비주거용,주거용
1819,11440-1515802,2,집합,3,표제부,서울특별시 마포구 노고산동 107-77번지,서울특별시마포구 서강로16길 45,정호오피스텔,11440,11000.0,0.0,107.0,77.0,NaN,NaN,NaN,0,1.144041e+11,11001.0,0,45.0,0.0,NaN,0.0,주건축물,186.10,110.46,59.36,672.98,550.38,295.74,21.0,철근콘크리트구조,철근콘크리트조,14000,업무시설,업무시설(오피스텔),10.0,(철근)콘크리트,평스라브,0,0,15.40,5,1,0,0,0,0.0,672.98,0,0.0,0,0.0,6,93.94,0,0.00,20010523,20010607,20011116.0,NaN,NaN,NaN,NaN,NaN,17,NaN,0.0,0,NaN,0,NaN,0,20111117,비주거용,주거용
3096,11560-844602,2,집합,3,표제부,서울특별시 영등포구 당산동2가 15-1번지,서울특별시영등포구 영등포로 112,문래역 대우미래사랑3차,11560,11200.0,0.0,15.0,1.0,NaN,NaN,NaN,0,1.156031e+11,11207.0,0,112.0,0.0,101동,0.0,주건축물,0.00,674.05,0.00,10871.52,8971.16,0.00,21.0,철근콘크리트구조,철근콘크리트구조,14000,업무시설,업무시설(오피스텔),10.0,(철근)콘크리트,(철근)콘크리트,0,0,44.95,15,2,2,0,0,0.0,10871.52,0,0.0,0,0.0,0,0.00,0,0.00,20020314,20020511,20040213.0,2002,3180081.0,건축과,1101.0,신축허가,82,NaN,0.0,0,NaN,0,NaN,0,20111117,비주거용,주거용


In [80]:
officetel_df['주_용도_코드_명'].value_counts()

업무시설         970
공동주택         252
제2종근린생활시설     15
제1종근린생활시설      4
판매시설           1
단독주택           1
숙박시설           1
Name: 주_용도_코드_명, dtype: int64

In [81]:
prac_df.head()

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주용도_주거용,기타용도_주거용
0,11305-10019310902,2,집합,3,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1,1.130541e+11,10301.0,0,36.0,0.0,삼일교회,0.0,주건축물,393.00,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),04000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3,0,12.2,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.00,7,80.50,20070821,20070829,20101216.0,2007,3080079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101228,비주거용,주거용
1,11440-2163902,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0,61.0,10.0,NaN,0.0,주건축물,177.62,105.30,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.00,4,46.00,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,주거용,주거용
2,11200-2360202,1,일반,2,일반건축물,서울특별시 성동구 송정동 73-566번지,서울특별시성동구 송정14길 17,대한예수교장로회비전교회,11200,11800.0,0.0,73.0,566.0,NaN,NaN,NaN,0,1.120041e+11,11801.0,0,17.0,0.0,NaN,0.0,주건축물,806.00,376.05,46.66,1185.07,797.98,99.00,21.0,철근콘크리트구조,"철근콘크리트,철골조",06000,종교시설,교회,10.0,(철근)콘크리트,"슬라브,경량철골조",0,0,13.2,2,1,0,0,0,0.0,1185.07,0,0.0,0,0.0,0,0.00,0,0.00,NaN,NaN,19781205.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110131,비주거용,비주거용
3,11380-2690602,1,일반,2,일반건축물,서울특별시 은평구 응암동 594-87번지,서울특별시은평구 응암로 163-13,NaN,11380,10700.0,0.0,594.0,87.0,NaN,NaN,NaN,0,1.138030e+11,10702.0,0,163.0,13.0,NaN,0.0,주건축물,0.00,0.00,0.00,246.15,0.00,0.00,11.0,벽돌구조,연와조,01000,단독주택,"단독주택,다가구용 (6가구)",10.0,(철근)콘크리트,슬라브,0,6,0.0,2,1,0,0,0,0.0,246.15,0,0.0,0,0.0,0,0.00,0,0.00,19900403,NaN,19911128.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110105,주거용,주거용
4,11110-10017892502,1,일반,2,일반건축물,서울특별시 종로구 평창동 365-1번지,서울특별시종로구 평창11길 45,NaN,11110,18300.0,0.0,365.0,1.0,NaN,NaN,NaN,0,1.111041e+11,18301.0,0,45.0,0.0,NaN,0.0,주건축물,356.00,104.22,29.28,404.80,194.31,54.58,21.0,철근콘크리트구조,철근콘크리트구조,03000,제1종근린생활시설,제1종근린생활시설,10.0,(철근)콘크리트,(철근)콘크리트,0,0,7.9,2,1,0,0,0,0.0,404.80,0,0.0,0,0.0,1,33.29,2,40.19,20070605,20070616,20080319.0,2007,3000082.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110101,비주거용,비주거용


In [82]:
prac_df.shape

(644134, 77)

In [83]:
prac_df['구'] = prac_df['대지_위치'].apply(lambda x: x.split(' ')[1])

In [84]:
prac_df.head()

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주용도_주거용,기타용도_주거용,구
0,11305-10019310902,2,집합,3,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1,1.130541e+11,10301.0,0,36.0,0.0,삼일교회,0.0,주건축물,393.00,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),04000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3,0,12.2,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.00,7,80.50,20070821,20070829,20101216.0,2007,3080079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101228,비주거용,주거용,강북구
1,11440-2163902,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0,61.0,10.0,NaN,0.0,주건축물,177.62,105.30,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.00,4,46.00,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,주거용,주거용,마포구
2,11200-2360202,1,일반,2,일반건축물,서울특별시 성동구 송정동 73-566번지,서울특별시성동구 송정14길 17,대한예수교장로회비전교회,11200,11800.0,0.0,73.0,566.0,NaN,NaN,NaN,0,1.120041e+11,11801.0,0,17.0,0.0,NaN,0.0,주건축물,806.00,376.05,46.66,1185.07,797.98,99.00,21.0,철근콘크리트구조,"철근콘크리트,철골조",06000,종교시설,교회,10.0,(철근)콘크리트,"슬라브,경량철골조",0,0,13.2,2,1,0,0,0,0.0,1185.07,0,0.0,0,0.0,0,0.00,0,0.00,NaN,NaN,19781205.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110131,비주거용,비주거용,성동구
3,11380-2690602,1,일반,2,일반건축물,서울특별시 은평구 응암동 594-87번지,서울특별시은평구 응암로 163-13,NaN,11380,10700.0,0.0,594.0,87.0,NaN,NaN,NaN,0,1.138030e+11,10702.0,0,163.0,13.0,NaN,0.0,주건축물,0.00,0.00,0.00,246.15,0.00,0.00,11.0,벽돌구조,연와조,01000,단독주택,"단독주택,다가구용 (6가구)",10.0,(철근)콘크리트,슬라브,0,6,0.0,2,1,0,0,0,0.0,246.15,0,0.0,0,0.0,0,0.00,0,0.00,19900403,NaN,19911128.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110105,주거용,주거용,은평구
4,11110-10017892502,1,일반,2,일반건축물,서울특별시 종로구 평창동 365-1번지,서울특별시종로구 평창11길 45,NaN,11110,18300.0,0.0,365.0,1.0,NaN,NaN,NaN,0,1.111041e+11,18301.0,0,45.0,0.0,NaN,0.0,주건축물,356.00,104.22,29.28,404.80,194.31,54.58,21.0,철근콘크리트구조,철근콘크리트구조,03000,제1종근린생활시설,제1종근린생활시설,10.0,(철근)콘크리트,(철근)콘크리트,0,0,7.9,2,1,0,0,0,0.0,404.80,0,0.0,0,0.0,1,33.29,2,40.19,20070605,20070616,20080319.0,2007,3000082.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110101,비주거용,비주거용,종로구


In [85]:
prac_df['동'] = prac_df['대지_위치'].apply(lambda x: x.split(' ')[2])

In [86]:
gaepo_df = prac_df[prac_df['동'] == '개포동']
print(gaepo_df.shape)

(1778, 79)


In [87]:
gaepo_df.head()

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주용도_주거용,기타용도_주거용,구,동
2681,11680-258702,1,일반,2,일반건축물,서울특별시 강남구 개포동 173번지,서울특별시강남구 개포로 402,개포고등학교,11680,10300.0,0.0,173.0,0.0,NaN,NaN,NaN,0,1.168031e+11,10301.0,0,402.0,0.0,교사동,0.0,주건축물,20145.8,3283.44,0.00,6824.00,8954.59,0.00,21.0,철근콘크리트구조,철근콘크리트조,10000,교육연구시설,교육연구시설,10.0,(철근)콘크리트,평슬라브,0,0,0.00,4,1,0,0,2,80.1,6824.00,0,0.0,0,0.0,0,0.0,24,276.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111117,비주거용,비주거용,강남구,개포동
2715,11680-272902,1,일반,2,일반건축물,서울특별시 강남구 개포동 1222-10번지,서울특별시강남구 논현로16길 14-5,NaN,11680,10300.0,0.0,1222.0,10.0,NaN,NaN,NaN,0,1.168042e+11,10301.0,0,14.0,5.0,NaN,0.0,주건축물,0.0,125.28,0.00,538.13,452.92,0.00,21.0,철근콘크리트구조,철근콘크리트라멘조 연와조,03000,제1종근린생활시설,근린생활시설 주거시설,10.0,(철근)콘크리트,슬라브,0,1,0.00,4,1,0,0,0,0.0,538.13,0,0.0,0,0.0,0,0.0,0,0.0,19880216,NaN,19880810.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111117,비주거용,주거용,강남구,개포동
2792,11680-779502,1,일반,2,일반건축물,서울특별시 강남구 개포동 1243-3번지,서울특별시강남구 개포로17길 19,NaN,11680,10300.0,0.0,1243.0,3.0,NaN,NaN,NaN,0,1.168042e+11,10301.0,0,19.0,0.0,NaN,0.0,주건축물,296.1,167.07,56.42,654.96,491.22,165.90,21.0,철근콘크리트구조,철근콘크리트조,01000,단독주택,주거시설,10.0,(철근)콘크리트,평슬라브,0,16,10.35,3,1,0,0,0,0.0,654.96,0,0.0,0,0.0,0,0.0,5,0.0,19950911,19950921,19960909.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111117,주거용,주거용,강남구,개포동
2837,11680-1873102,2,집합,3,표제부,서울특별시 강남구 개포동 1202-2번지,서울특별시강남구 개포로22길 41-6,NaN,11680,10300.0,0.0,1202.0,2.0,NaN,NaN,NaN,0,1.168042e+11,10301.0,0,41.0,6.0,NaN,0.0,주건축물,210.0,113.46,54.03,562.34,448.88,213.75,21.0,철근콘크리트구조,철근콘크리트조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,슬라브,10,0,12.60,4,1,0,0,0,0.0,562.34,0,0.0,0,0.0,0,0.0,4,0.0,19910320,NaN,19911017.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111117,주거용,주거용,강남구,개포동
3064,11680-2226102,2,집합,3,표제부,서울특별시 강남구 개포동 1168-11번지,서울특별시강남구 논현로2길 11,다세대주택,11680,10300.0,0.0,1168.0,11.0,NaN,NaN,NaN,0,1.168042e+11,10301.0,0,11.0,0.0,NaN,0.0,주건축물,277.1,161.21,58.18,654.76,520.20,187.73,21.0,철근콘크리트구조,철근콘크리트조 연와조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,평슬라브,9,0,0.00,4,1,0,0,0,0.0,654.76,0,0.0,0,0.0,0,0.0,5,0.0,19921211,1993,19930906.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111117,주거용,주거용,강남구,개포동


In [88]:
gaepo_1262_df = gaepo_df[gaepo_df['번'] == 1262]
gaepo_1262_df.shape

(17, 79)

In [89]:
gaepo_1262_df

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주용도_주거용,기타용도_주거용,구,동
58105,11680-423902,1,일반,2,일반건축물,서울특별시 강남구 개포동 1262-9번지,서울특별시강남구 개포로 257,NaN,11680,10300.0,0.0,1262.0,9.0,NaN,NaN,NaN,0,1.168031e+11,10301.0,0,257.0,0.0,B동(증측동),0.0,주건축물,746.80,224.89,30.11,1371.95,916.45,122.72,21.0,철근콘크리트구조,철근콘크리트조,11000,노유자시설,주거시설 종교시설 노유자시설,10.0,(철근)콘크리트,평슬라브,0,1,21.90,5,2,0,0,0,0.0,1371.95,0,0.0,0,0.0,6,0.00,9,0.0,NaN,NaN,19971210.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20140123,비주거용,주거용,강남구,개포동
134100,11680-1895202,1,일반,2,일반건축물,서울특별시 강남구 개포동 1262-11번지,서울특별시강남구 개포로 253,NaN,11680,10300.0,0.0,1262.0,11.0,NaN,NaN,NaN,0,1.168031e+11,10301.0,0,253.0,0.0,NaN,0.0,주건축물,314.60,154.45,49.09,533.38,302.55,96.17,21.0,철근콘크리트구조,철근콘크리트조,03000,제1종근린생활시설,근린생활시설,10.0,(철근)콘크리트,평슬라브,0,0,8.00,3,1,0,0,0,0.0,533.38,0,0.0,0,0.0,0,0.00,4,46.0,19910121.0,19910315.0,19910805.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111123,비주거용,비주거용,강남구,개포동
144183,11680-1478602,1,일반,2,일반건축물,서울특별시 강남구 개포동 1262-16번지,서울특별시강남구 개포로31길 12,NaN,11680,10300.0,0.0,1262.0,16.0,NaN,NaN,NaN,0,1.168042e+11,10301.0,0,12.0,0.0,NaN,0.0,주건축물,241.70,120.54,49.87,120.54,120.54,49.87,11.0,벽돌구조,조적조 목조,04000,제2종근린생활시설,근린생활시설,90.0,기타지붕,트러스,0,0,4.90,1,0,0,0,0,0.0,120.54,0,0.0,0,0.0,0,0.00,0,0.0,19920918.0,19921103,19921229.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111123,비주거용,비주거용,강남구,개포동
144225,11680-1580602,1,일반,2,일반건축물,서울특별시 강남구 개포동 1262-17번지,서울특별시강남구 개포로31길 14,NaN,11680,10300.0,0.0,1262.0,17.0,NaN,NaN,NaN,0,1.168042e+11,10301.0,0,14.0,0.0,NaN,0.0,주건축물,216.20,108.00,49.95,418.94,310.94,143.82,21.0,철근콘크리트구조,철근콘크리트조 연와조,01000,단독주택,주거시설,90.0,기타지붕,스페니쉬기와,0,7,9.40,3,1,0,0,0,0.0,418.94,0,0.0,0,0.0,0,0.00,2,0.0,19900702.0,19900723,19901121.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111123,주거용,주거용,강남구,개포동
255825,11680-1965302,1,일반,2,일반건축물,서울특별시 강남구 개포동 1262-15번지,서울특별시강남구 개포로31길 10,NaN,11680,10300.0,0.0,1262.0,15.0,NaN,NaN,NaN,0,1.168042e+11,10301.0,0,10.0,0.0,NaN,0.0,주건축물,264.50,115.88,44.81,261.49,213.46,80.70,11.0,벽돌구조,연와조,01000,단독주택,주거시설,10.0,(철근)콘크리트,슬라브위기와,0,1,7.90,2,1,0,0,0,0.0,261.49,0,0.0,0,0.0,1,0.00,0,0.0,19890329.0,19890331.0,19891025.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111124,주거용,주거용,강남구,개포동
304719,11680-422002,2,집합,3,표제부,서울특별시 강남구 개포동 1262-6번지,서울특별시강남구 개포로35길 9,NaN,11680,10300.0,0.0,1262.0,6.0,NaN,NaN,NaN,0,1.168042e+11,10301.0,0,9.0,0.0,NaN,0.0,주건축물,216.50,102.12,47.17,306.36,204.24,94.34,11.0,벽돌구조,연와조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,슬라브,6,0,7.80,2,1,0,0,0,0.0,306.36,0,0.0,0,0.0,0,0.00,2,0.0,19891025.0,19890327.0,19891118.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111117,주거용,주거용,강남구,개포동
314641,11680-844702,2,집합,3,표제부,서울특별시 강남구 개포동 1262-1번지,서울특별시강남구 개포로31길 18,NaN,11680,10300.0,0.0,1262.0,1.0,NaN,NaN,NaN,0,1.168042e+11,10301.0,0,18.0,0.0,NaN,0.0,주건축물,264.30,123.68,46.80,616.00,492.80,186.45,21.0,철근콘크리트구조,철근콘크리트조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,슬라브,9,0,11.20,4,1,0,0,0,0.0,616.00,0,0.0,0,0.0,2,0.00,2,0.0,19910615,19910720,19920418.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20111117,주거용,주거용,강남구,개포동
441162,11680-332202,1,일반,2,일반건축물,서울특별시 강남구 개포동 1262-10번지,서울특별시강남구 개포로 255,NaN,11680,10300.0,0.0,1262.0,10.0,NaN,NaN,NaN,0,1.168031e+11,10301.0,0,255.0,0.0,NaN,0.0,주건축물,343.50,165.39,48.15,997.07,801.95,233.46,21.0,철근콘크리트구조,철근콘크리트조,04000,제2종근린생활시설,"근린생활시설,단독주택",10.0,(철근)콘크리트,슬라브,0,1,17.70,5,1,0,0,0,0.0,997.07,0,0.0,0,0.0,0,0.00,6,0.0,19880616,19890829.0,19900

In [90]:
gaepo_1262_df['지'].value_counts()

9.0     2
14.0    1
15.0    1
4.0     1
12.0    1
11.0    1
17.0    1
5.0     1
1.0     1
7.0     1
2.0     1
16.0    1
6.0     1
0.0     1
10.0    1
3.0     1
Name: 지, dtype: int64

In [91]:
gaepo_1262_df['지'].nunique()

16

In [92]:
gaepo_1262_df['지'].max()

17.0

In [93]:
main_data_df = pd.read_csv('./prepped_data/main_data_ver_2.4.csv')
print(main_data_df.shape)
main_data_df.head()

(4559866, 16)


,지번주소,도로명주소,대장구분명,대장구분코드,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건축년도,건물나이,target_log_transformed,target/area_log_transformed
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,9,1988,25,10.950807,6.597308
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,12,1988,25,10.950807,6.597308
2,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,55000,0,817.479191,5,2013,2,1987,26,10.915088,6.706225
3,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,58250,0,865.784780,4,2013,2,1987,26,10.972499,6.763636
4,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,60000,0,891.795482,5,2013,5,1987,26,11.002100,6.793237


In [95]:
prac_df['주소'] = prac_df['대지_위치'].str.replace('번지', '')

In [97]:
prac_df['split_length'] = prac_df['주소'].apply(lambda x: len(x.split('-')))

In [98]:
prac_df['split_length'].value_counts()

2    566491
1     77627
3        16
Name: split_length, dtype: int64

In [100]:
split_1_df = prac_df[prac_df['split_length'] == 1]
print(split_1_df.shape)
split_1_df.head()

(77627, 81)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주용도_주거용,기타용도_주거용,구,동,주소,split_length
11,11740-10017967102,2,집합,3,표제부,서울특별시 강동구 강일동 674번지,서울특별시강동구 아리수로93길 40,강일리버파크,11740,11000.0,0.0,674.0,0.0,NaN,NaN,NaN,0,1.174042e+11,11001.0,0,40.0,0.0,301동,0.0,주건축물,0.0,564.01,0.0,6401.19,6401.19,0.00,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,공동주택(아파트),10.0,(철근)콘크리트,(철근)콘크리트,78,0,45.60,14,0,3,0,0,0.0,6401.19,0,0.0,0,0.0,0,0.0,0,0.0,20041224,20060926,20090311.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110122,주거용,주거용,강동구,강일동,서울특별시 강동구 강일동 674,1
12,11110-2843902,1,일반,2,일반건축물,서울특별시 종로구 구기동 84번지,서울특별시종로구 진흥로21길 9-3,NaN,11110,18200.0,0.0,84.0,0.0,NaN,NaN,NaN,0,1.111041e+11,18201.0,0,9.0,3.0,NaN,0.0,주건축물,314.0,81.33,25.9,177.53,136.95,43.61,11.0,벽돌구조,연와조,01000,단독주택,주택,10.0,(철근)콘크리트,경사스라브,0,1,7.55,2,1,0,0,0,0.0,177.53,0,0.0,0,0.0,1,22.5,0,0.0,NaN,NaN,19731026.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110419,주거용,주거용,종로구,구기동,서울특별시 종로구 구기동 84,1
14,11170-464202,1,일반,2,일반건축물,서울특별시 용산구 서계동 57번지,서울특별시용산구 청파로77길 34,NaN,11170,10800.0,0.0,57.0,0.0,NaN,NaN,NaN,0,1.117041e+11,10801.0,0,34.0,0.0,NaN,0.0,주건축물,0.0,0.00,0.0,98.12,0.00,0.00,11.0,벽돌구조,연와조,01000,단독주택,주택,10.0,(철근)콘크리트,평옥개,0,1,0.00,2,0,0,0,0,0.0,98.12,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19660320.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20100729,주거용,주거용,용산구,서계동,서울특별시 용산구 서계동 57,1
42,11290-10019314002,2,집합,3,표제부,서울특별시 성북구 삼선동2가 420번지,서울특별시성북구 낙산길 243-15,삼선현대힐스테이트,11290,11200.0,0.0,420.0,0.0,NaN,NaN,NaN,0,1.129041e+11,11201.0,0,243.0,15.0,지하주차장,1.0,부속건축물,0.0,0.00,0.0,14309.28,0.00,0.00,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,지하주차장,10.0,(철근)콘크리트,(철근)콘크리트,0,0,0.00,0,2,0,0,0,0.0,14309.28,0,0.0,0,0.0,0,0.0,0,0.0,20020916,20040306,20091201.0,2002,3070087.0,도시개발과,2101.0,주택건설사업계획승인,0,NaN,0.0,0,NaN,0,NaN,0,20100325,주거용,비주거용,성북구,삼선동2가,서울특별시 성북구 삼선동2가 420,1
57,11320-1697802,2,집합,3,표제부,서울특별시 도봉구 도봉동 655번지,서울특별시도봉구 도봉로170길 40,래미안도봉아파트,11320,10800.0,0.0,655.0,0.0,NaN,NaN,NaN,0,1.132041e+11,10801.0,0,40.0,0.0,107동,0.0,주건축물,0.0,240.90,0.0,3054.16,3054.16,0.00,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,아파트,10.0,(철근)콘크리트,"(철근)콘크리트,경사지붕",29,0,45.59,16,0,0,1,0,0.0,3054.16,0,0.0,0,0.0,0,0.0,0,0.0,20020208,20021226,20050218.0,2002,3090076.0,도시정비과,2101.0,주택건설사업계획승인,0,NaN,0.0,0,NaN,0,NaN,0,20100417,주거용,주거용,도봉구,도봉동,서울특별시 도봉구 도봉동 655,1


In [99]:
split_3_df = prac_df[prac_df['split_length'] == 3]
print(split_3_df.shape)
split_3_df.head()

(16, 81)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주용도_주거용,기타용도_주거용,구,동,주소,split_length
8043,11530-10020647102,2,집합,3,표제부,서울특별시 구로구 천왕동 블록천왕1지구지구A-1-3블록,서울특별시구로구 천왕로1가길 9,NaN,11530,11100.0,2.0,0.0,0.0,천왕1지구,A-1-3,NaN,0,1.153041e+11,11101.0,0,9.0,NaN,4동,1.0,부속건축물,0.0,0.00,0.0000,954.24,0.00,0.0000,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,주차장(전체주차장),10.0,(철근)콘크리트,(철근)콘크리트,0,0,5.400,0,1,1,0,0,0.0,954.24,0,0.0,0,0.0,22,847.66,0,0.0,20120524,20120608,20121122.0,2012,3160173.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20140611,주거용,비주거용,구로구,천왕동,서울특별시 구로구 천왕동 블록천왕1지구지구A-1-3블록,3
27963,11380-10024422602,1,일반,2,일반건축물,서울특별시 은평구 진관동 블록은평뉴타운 단독주택용지 3-⑥-8블럭지구,서울특별시은평구 진관길 21-2,은평한옥마을 생활한옥 시범주택,11380,11400.0,2.0,0.0,0.0,은평뉴타운 단독주택용지 3-⑥-8블럭,NaN,NaN,0,1.138041e+11,11401.0,0,21.0,2.0,나동,1.0,부속건축물,330.0,13.23,4.0100,13.23,13.23,4.0100,51.0,일반목구조,일반목구조,01000,단독주택,단독주택(한옥주택),20.0,기와,기와(목구조),0,0,4.555,1,0,0,0,0,0.0,13.23,0,0.0,0,0.0,0,0.00,1,11.5,20130718.0,20130914,20140515.0,2013,3110120.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20140521,주거용,주거용,은평구,진관동,서울특별시 은평구 진관동 블록은평뉴타운 단독주택용지 3-⑥-8블럭지구,3
68317,11530-10020916202,2,집합,3,표제부,서울특별시 구로구 천왕동 블록A-1-1지구,서울특별시구로구 천왕로1길 36,천왕타운하우스1,11530,11100.0,2.0,0.0,0.0,A-1-1,NaN,NaN,0,1.153041e+11,11101.0,0,36.0,NaN,109,0.0,주건축물,0.0,182.57,0.0000,848.78,558.21,0.0000,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,"도시형생활주택(단지형다세대주택),8세대",90.0,기타지붕,기타지붕,8,0,14.500,4,1,1,0,0,0.0,848.78,0,0.0,0,0.0,0,0.00,0,0.0,20121010,20121024.0,20130523.0,2012,3160173.0,건축과,2101.0,주택건설사업계획승인,8,NaN,0.0,0,NaN,0,NaN,0,20140611,주거용,주거용,구로구,천왕동,서울특별시 구로구 천왕동 블록A-1-1지구,3
158564,11530-10020742002,2,집합,3,표제부,서울특별시 구로구 천왕동 블록A-1-2지구,서울특별시구로구 천왕로1길 24,천왕타운하우스2,11530,11100.0,2.0,0.0,0.0,A-1-2,NaN,NaN,0,1.153041e+11,11101.0,0,24.0,NaN,105동,0.0,주건축물,0.0,164.63,0.0000,864.30,559.89,0.0000,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,"도시형생활주택(단지형다세대주택,8세대)",90.0,기타지붕,기타지붕,8,0,15.100,4,1,1,0,0,0.0,864.30,0,0.0,0,0.0,0,0.00,0,0.0,20120808,20120816,20121228.0,2012,3160173.0,건축과,2101.0,주택건설사업계획승인,0,NaN,0.0,0,NaN,0,NaN,0,20140611,주거용,주거용,구로구,천왕동,서울특별시 구로구 천왕동 블록A-1-2지구,3
179736,11200-10018715202,1,일반,2,일반건축물,서울특별시 성동구 상왕십리동 블록왕십리 뉴타운 제2구역지구택지-4블록종교부지-2로트,서울특별시성동구 마장로19길 29,왕십리중앙교회,11200,10100.0,2.0,0.0,0.0,왕십리 뉴타운 제2구역,택지-4,종교부지-2,0,1.120041e+11,10101.0,0,29.0,0.0,NaN,0.0,주건축물,860.0,428.26,49.7977,4118.51,2138.05,248.6105,21.0,철근콘크리트구조,철근콘크리트구조,06000,종교시설,종교시설,10.0,(철근)콘크리트,(철근)콘크리트,0,0,41.600,8,3,1,0,0,0.0,4118.51,0,0.0,0,0.0,29,338.50,1,16.5,20090106,20110104,20110704.0,2009,3030079.0,주택과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20141015,비주거용,비주거용,성동구,상왕십리동,서울특별시 성동구 상왕십리동 블록왕십리 뉴타운 제2구역지구택지-4블록종교부지-2로트,3


In [96]:
prac_df.head()

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주용도_주거용,기타용도_주거용,구,동,주소
0,11305-10019310902,2,집합,3,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1,1.130541e+11,10301.0,0,36.0,0.0,삼일교회,0.0,주건축물,393.00,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),04000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3,0,12.2,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.00,7,80.50,20070821,20070829,20101216.0,2007,3080079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101228,비주거용,주거용,강북구,수유동,서울특별시 강북구 수유동 484-80
1,11440-2163902,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0,61.0,10.0,NaN,0.0,주건축물,177.62,105.30,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.00,4,46.00,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,주거용,주거용,마포구,서교동,서울특별시 마포구 서교동 484-16
2,11200-2360202,1,일반,2,일반건축물,서울특별시 성동구 송정동 73-566번지,서울특별시성동구 송정14길 17,대한예수교장로회비전교회,11200,11800.0,0.0,73.0,566.0,NaN,NaN,NaN,0,1.120041e+11,11801.0,0,17.0,0.0,NaN,0.0,주건축물,806.00,376.05,46.66,1185.07,797.98,99.00,21.0,철근콘크리트구조,"철근콘크리트,철골조",06000,종교시설,교회,10.0,(철근)콘크리트,"슬라브,경량철골조",0,0,13.2,2,1,0,0,0,0.0,1185.07,0,0.0,0,0.0,0,0.00,0,0.00,NaN,NaN,19781205.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110131,비주거용,비주거용,성동구,송정동,서울특별시 성동구 송정동 73-566
3,11380-2690602,1,일반,2,일반건축물,서울특별시 은평구 응암동 594-87번지,서울특별시은평구 응암로 163-13,NaN,11380,10700.0,0.0,594.0,87.0,NaN,NaN,NaN,0,1.138030e+11,10702.0,0,163.0,13.0,NaN,0.0,주건축물,0.00,0.00,0.00,246.15,0.00,0.00,11.0,벽돌구조,연와조,01000,단독주택,"단독주택,다가구용 (6가구)",10.0,(철근)콘크리트,슬라브,0,6,0.0,2,1,0,0,0,0.0,246.15,0,0.0,0,0.0,0,0.00,0,0.00,19900403,NaN,19911128.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110105,주거용,주거용,은평구,응암동,서울특별시 은평구 응암동 594-87
4,11110-10017892502,1,일반,2,일반건축물,서울특별시 종로구 평창동 365-1번지,서울특별시종로구 평창11길 45,NaN,11110,18300.0,0.0,365.0,1.0,NaN,NaN,NaN,0,1.111041e+11,18301.0,0,45.0,0.0,NaN,0.0,주건축물,356.00,104.22,29.28,404.80,194.31,54.58,21.0,철근콘크리트구조,철근콘크리트구조,03000,제1종근린생활시설,제1종근린생활시설,10.0,(철근)콘크리트,(철근)콘크리트,0,0,7.9,2,1,0,0,0,0.0,404.80,0,0.0,0,0.0,1,33.29,2,40.19,20070605,20070616,20080319.0,2007,3000082.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110101,비주거용,비주거용,종로구,평창동,서울특별시 종로구 평창동 365-1


In [101]:
def addr_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        splitted = x.split('-')
        if len(splitted) > 1:
            return x
        else:
            return x + '-0'

In [102]:
prac_df['주소_modded'] = prac_df['주소'].apply(addr_modifier)
prac_df.head()

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주용도_주거용,기타용도_주거용,구,동,주소,split_length,주소_modded
0,11305-10019310902,2,집합,3,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1,1.130541e+11,10301.0,0,36.0,0.0,삼일교회,0.0,주건축물,393.00,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),04000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3,0,12.2,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.00,7,80.50,20070821,20070829,20101216.0,2007,3080079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101228,비주거용,주거용,강북구,수유동,서울특별시 강북구 수유동 484-80,2,서울특별시 강북구 수유동 484-80
1,11440-2163902,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0,61.0,10.0,NaN,0.0,주건축물,177.62,105.30,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.00,4,46.00,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,주거용,주거용,마포구,서교동,서울특별시 마포구 서교동 484-16,2,서울특별시 마포구 서교동 484-16
2,11200-2360202,1,일반,2,일반건축물,서울특별시 성동구 송정동 73-566번지,서울특별시성동구 송정14길 17,대한예수교장로회비전교회,11200,11800.0,0.0,73.0,566.0,NaN,NaN,NaN,0,1.120041e+11,11801.0,0,17.0,0.0,NaN,0.0,주건축물,806.00,376.05,46.66,1185.07,797.98,99.00,21.0,철근콘크리트구조,"철근콘크리트,철골조",06000,종교시설,교회,10.0,(철근)콘크리트,"슬라브,경량철골조",0,0,13.2,2,1,0,0,0,0.0,1185.07,0,0.0,0,0.0,0,0.00,0,0.00,NaN,NaN,19781205.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110131,비주거용,비주거용,성동구,송정동,서울특별시 성동구 송정동 73-566,2,서울특별시 성동구 송정동 73-566
3,11380-2690602,1,일반,2,일반건축물,서울특별시 은평구 응암동 594-87번지,서울특별시은평구 응암로 163-13,NaN,11380,10700.0,0.0,594.0,87.0,NaN,NaN,NaN,0,1.138030e+11,10702.0,0,163.0,13.0,NaN,0.0,주건축물,0.00,0.00,0.00,246.15,0.00,0.00,11.0,벽돌구조,연와조,01000,단독주택,"단독주택,다가구용 (6가구)",10.0,(철근)콘크리트,슬라브,0,6,0.0,2,1,0,0,0,0.0,246.15,0,0.0,0,0.0,0,0.00,0,0.00,19900403,NaN,19911128.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110105,주거용,주거용,은평구,응암동,서울특별시 은평구 응암동 594-87,2,서울특별시 은평구 응암동 594-87
4,11110-10017892502,1,일반,2,일반건축물,서울특별시 종로구 평창동 365-1번지,서울특별시종로구 평창11길 45,NaN,11110,18300.0,0.0,365.0,1.0,NaN,NaN,NaN,0,1.111041e+11,18301.0,0,45.0,0.0,NaN,0.0,주건축물,356.00,104.22,29.28,404.80,194.31,54.58,21.0,철근콘크리트구조,철근콘크리트구조,03000,제1종근린생활시설,제1종근린생활시설,10.0,(철근)콘크리트,(철근)콘크리트,0,0,7.9,2,1,0,0,0,0.0,404.80,0,0.0,0,0.0,1,33.29,2,40.19,20070605,20070616,20080319.0,2007,3000082.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110101,비주거용,비주거용,종로구,평창동,서울특별시 종로구 평창동 365-1,2,서울특별시 종로구 평창동 365-1


In [105]:
main_data_df.shape

(4559866, 16)

In [103]:
intersection_df = main_data_df[main_data_df['지번주소'].isin(prac_df['주소_modded'].unique())]
print(intersection_df.shape)
intersection_df.head()

(4326844, 16)


,지번주소,도로명주소,대장구분명,대장구분코드,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건축년도,건물나이,target_log_transformed,target/area_log_transformed
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,9,1988,25,10.950807,6.597308
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,12,1988,25,10.950807,6.597308
2,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,55000,0,817.479191,5,2013,2,1987,26,10.915088,6.706225
3,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,58250,0,865.784780,4,2013,2,1987,26,10.972499,6.763636
4,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,60000,0,891.795482,5,2013,5,1987,26,11.002100,6.793237


In [106]:
filenames

['shared_cols_bdinfo_2014.csv',
 'shared_cols_bdinfo_2015.csv',
 'shared_cols_bdinfo_2016.csv',
 'shared_cols_bdinfo_2017.csv',
 'shared_cols_bdinfo_2018.csv',
 'shared_cols_bdinfo_2019.csv',
 'shared_cols_bdinfo_2020.csv']

In [108]:
dfs_list = []
for f in tqdm(filenames):
    df = pd.read_csv(basedir + f)
    dfs_list.append(df)
concat_df = pd.concat(dfs_list)
print(concat_df.shape)
concat_df.head()

  0%|                                                                                            | 0/7 [00:00<?, ?it/s]C:\Users\나\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (14,15,58,59) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 14%|████████████                                                                        | 1/7 [00:11<01:10, 11.69s/it]C:\Users\나\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (13,14,15,34,58,59,67) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 29%|████████████████████████                                                            | 2/7 [00:19<00:53, 10.65s/it]C:\Users\나\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3147

(4367214, 75)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,11305-10019310902,2,집합,3,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1,1.130541e+11,10301.0,0.0,36.0,0.0,삼일교회,0.0,주건축물,393.00,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),04000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3,0,12.2,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.00,7,80.50,20070821,20070829,20101216.0,2007,3080079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101228
1,11440-2163902,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.30,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.00,4,46.00,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230
2,11200-2360202,1,일반,2,일반건축물,서울특별시 성동구 송정동 73-566번지,서울특별시성동구 송정14길 17,대한예수교장로회비전교회,11200,11800.0,0.0,73.0,566.0,NaN,NaN,NaN,0,1.120041e+11,11801.0,0.0,17.0,0.0,NaN,0.0,주건축물,806.00,376.05,46.66,1185.07,797.98,99.00,21.0,철근콘크리트구조,"철근콘크리트,철골조",06000,종교시설,교회,10.0,(철근)콘크리트,"슬라브,경량철골조",0,0,13.2,2,1,0,0,0,0.0,1185.07,0,0.0,0,0.0,0,0.00,0,0.00,NaN,NaN,19781205.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110131
3,11380-2690602,1,일반,2,일반건축물,서울특별시 은평구 응암동 594-87번지,서울특별시은평구 응암로 163-13,NaN,11380,10700.0,0.0,594.0,87.0,NaN,NaN,NaN,0,1.138030e+11,10702.0,0.0,163.0,13.0,NaN,0.0,주건축물,0.00,0.00,0.00,246.15,0.00,0.00,11.0,벽돌구조,연와조,01000,단독주택,"단독주택,다가구용 (6가구)",10.0,(철근)콘크리트,슬라브,0,6,0.0,2,1,0,0,0,0.0,246.15,0,0.0,0,0.0,0,0.00,0,0.00,19900403,NaN,19911128.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110105
4,11110-10017892502,1,일반,2,일반건축물,서울특별시 종로구 평창동 365-1번지,서울특별시종로구 평창11길 45,NaN,11110,18300.0,0.0,365.0,1.0,NaN,NaN,NaN,0,1.111041e+11,18301.0,0.0,45.0,0.0,NaN,0.0,주건축물,356.00,104.22,29.28,404.80,194.31,54.58,21.0,철근콘크리트구조,철근콘크리트구조,03000,제1종근린생활시설,제1종근린생활시설,10.0,(철근)콘크리트,(철근)콘크리트,0,0,7.9,2,1,0,0,0,0.0,404.80,0,0.0,0,0.0,1,33.29,2,40.19,20070605,20070616,20080319.0,2007,3000082.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110101


In [109]:
concat_df['주소'] = concat_df['대지_위치'].str.replace('번지', '')
concat_df['주소'] = concat_df['주소'].apply(addr_modifier)
concat_df.head(1)

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주소
0,11305-10019310902,2,집합,3,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1,1.130541e+11,10301.0,0.0,36.0,0.0,삼일교회,0.0,주건축물,393.0,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),04000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3,0,12.2,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.0,7,80.5,20070821,20070829,20101216.0,2007,3080079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101228,서울특별시 강북구 수유동 484-80


In [111]:
main_data_df.shape

(4559866, 16)

In [110]:
intersection_df = main_data_df[main_data_df['지번주소'].isin(concat_df['주소'].unique())]
print(intersection_df.shape)

(4530674, 16)


In [112]:
not_in_df = main_data_df[main_data_df['지번주소'].isin(concat_df['주소'].unique()) == False]
print(not_in_df.shape)
not_in_df.head()

(29192, 16)


,지번주소,도로명주소,대장구분명,대장구분코드,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건축년도,건물나이,target_log_transformed,target/area_log_transformed
14598,서울특별시 강남구 논현동 276-0,서울특별시 강남구 언주로 604-0,일반,1,아파트_매매,95.48,52500,0,549.853372,10,2006,1,1978,28,10.868568,6.309652
14599,서울특별시 강남구 논현동 276-0,서울특별시 강남구 언주로 604-0,일반,1,아파트_매매,95.48,55000,0,576.036866,6,2006,1,1978,28,10.915088,6.356172
14600,서울특별시 강남구 논현동 276-0,서울특별시 강남구 언주로 604-0,일반,1,아파트_매매,95.48,54000,0,565.563469,7,2006,1,1978,28,10.896739,6.337823
14601,서울특별시 강남구 논현동 276-0,서울특별시 강남구 언주로 604-0,일반,1,아파트_매매,95.48,55500,0,581.273565,12,2006,2,1978,28,10.924138,6.365221
14602,서울특별시 강남구 논현동 276-0,서울특별시 강남구 언주로 604-0,일반,1,아파트_매매,128.67,79000,0,613.973731,10,2006,2,1978,28,11.277203,6.419952


In [113]:
not_in_df['지번주소'].nunique()

1099

In [115]:
not_in_df['지번주소'].unique()

array(['서울특별시 강남구 논현동 276-0', '서울특별시 강남구 대치동 633-0',
       '서울특별시 강남구 대치동 610-0', ..., '서울특별시 중랑구 묵동 302-4',
       '서울특별시 중랑구 묵동 242-91', '서울특별시 구로구 오류동 306-4'], dtype=object)

In [114]:
not_in_df['trade_type'].unique()

array(['아파트_매매', '아파트_전세', '아파트_월세', '다세대연립_매매', '다세대연립_전세', '다세대연립_월세',
       '오피스텔_매매', '오피스텔_전세', '오피스텔_월세'], dtype=object)

In [117]:
bubun_0_df = concat_df[concat_df['지'] == 0]
print(bubun_0_df.shape)
bubun_0_df.head()

(547063, 76)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주소
11,11740-10017967102,2,집합,3,표제부,서울특별시 강동구 강일동 674번지,서울특별시강동구 아리수로93길 40,강일리버파크,11740,11000.0,0.0,674.0,0.0,NaN,NaN,NaN,0,1.174042e+11,11001.0,0.0,40.0,0.0,301동,0.0,주건축물,0.0,564.01,0.0,6401.19,6401.19,0.00,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,공동주택(아파트),10.0,(철근)콘크리트,(철근)콘크리트,78,0,45.60,14,0,3,0,0,0.0,6401.19,0,0.0,0,0.0,0,0.0,0,0.0,20041224,20060926,20090311.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110122,서울특별시 강동구 강일동 674-0
12,11110-2843902,1,일반,2,일반건축물,서울특별시 종로구 구기동 84번지,서울특별시종로구 진흥로21길 9-3,NaN,11110,18200.0,0.0,84.0,0.0,NaN,NaN,NaN,0,1.111041e+11,18201.0,0.0,9.0,3.0,NaN,0.0,주건축물,314.0,81.33,25.9,177.53,136.95,43.61,11.0,벽돌구조,연와조,01000,단독주택,주택,10.0,(철근)콘크리트,경사스라브,0,1,7.55,2,1,0,0,0,0.0,177.53,0,0.0,0,0.0,1,22.5,0,0.0,NaN,NaN,19731026.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110419,서울특별시 종로구 구기동 84-0
14,11170-464202,1,일반,2,일반건축물,서울특별시 용산구 서계동 57번지,서울특별시용산구 청파로77길 34,NaN,11170,10800.0,0.0,57.0,0.0,NaN,NaN,NaN,0,1.117041e+11,10801.0,0.0,34.0,0.0,NaN,0.0,주건축물,0.0,0.00,0.0,98.12,0.00,0.00,11.0,벽돌구조,연와조,01000,단독주택,주택,10.0,(철근)콘크리트,평옥개,0,1,0.00,2,0,0,0,0,0.0,98.12,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19660320.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20100729,서울특별시 용산구 서계동 57-0
42,11290-10019314002,2,집합,3,표제부,서울특별시 성북구 삼선동2가 420번지,서울특별시성북구 낙산길 243-15,삼선현대힐스테이트,11290,11200.0,0.0,420.0,0.0,NaN,NaN,NaN,0,1.129041e+11,11201.0,0.0,243.0,15.0,지하주차장,1.0,부속건축물,0.0,0.00,0.0,14309.28,0.00,0.00,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,지하주차장,10.0,(철근)콘크리트,(철근)콘크리트,0,0,0.00,0,2,0,0,0,0.0,14309.28,0,0.0,0,0.0,0,0.0,0,0.0,20020916,20040306,20091201.0,2002,3070087.0,도시개발과,2101.0,주택건설사업계획승인,0,NaN,0.0,0,NaN,0,NaN,0,20100325,서울특별시 성북구 삼선동2가 420-0
57,11320-1697802,2,집합,3,표제부,서울특별시 도봉구 도봉동 655번지,서울특별시도봉구 도봉로170길 40,래미안도봉아파트,11320,10800.0,0.0,655.0,0.0,NaN,NaN,NaN,0,1.132041e+11,10801.0,0.0,40.0,0.0,107동,0.0,주건축물,0.0,240.90,0.0,3054.16,3054.16,0.00,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,아파트,10.0,(철근)콘크리트,"(철근)콘크리트,경사지붕",29,0,45.59,16,0,0,1,0,0.0,3054.16,0,0.0,0,0.0,0,0.0,0,0.0,20020208,20021226,20050218.0,2002,3090076.0,도시정비과,2101.0,주택건설사업계획승인,0,NaN,0.0,0,NaN,0,NaN,0,20100417,서울특별시 도봉구 도봉동 655-0


In [118]:
concat_df.shape

(4367214, 76)

In [119]:
inter2_df = concat_df[concat_df['주소'].isin(main_data_df['지번주소'].unique())]
print(inter2_df.shape)
inter2_df.head()

(954843, 76)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주소
1,11440-2163902,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.300,59.28,393.130,385.210,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.00,4,1,0,0,0,0.0,393.130,0,0.0,0,0.0,0,0.00,4,46.0,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,서울특별시 마포구 서교동 484-16
6,11590-10018266302,2,집합,3,표제부,서울특별시 동작구 상도동 302-28번지,서울특별시동작구 국사봉8길 64-17,두성쉐르빌,11590,10200.0,0.0,302.0,28.0,NaN,NaN,NaN,1,1.159042e+11,10201.0,0.0,64.0,17.0,NaN,0.0,주건축물,389.48,232.420,59.67,947.740,659.890,169.43,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트,10,0,11.26,4,1,0,0,0,0.0,947.740,0,0.0,0,0.0,8,287.85,2,11.5,20080404,20080405,20080826.0,2008,3190077.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110119,서울특별시 동작구 상도동 302-28
7,11320-10018224102,2,집합,3,표제부,서울특별시 도봉구 쌍문동 380-24번지,서울특별시도봉구 노해로40길 25,NaN,11320,10500.0,0.0,380.0,24.0,NaN,NaN,NaN,0,1.132041e+11,10501.0,0.0,25.0,0.0,NaN,0.0,주건축물,208.00,81.085,38.98,268.165,268.165,128.93,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트,4,0,10.40,4,0,0,0,0,0.0,268.165,0,0.0,0,0.0,3,0.00,1,0.0,20080318,20080321,20080721.0,2008,3090077.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110202,서울특별시 도봉구 쌍문동 380-24
8,11440-3217402,2,집합,3,표제부,서울특별시 마포구 성산동 200-260번지,서울특별시마포구 월드컵북로27길 29,대현하이빌,11440,12500.0,0.0,200.0,260.0,NaN,NaN,NaN,0,1.144041e+11,12501.0,0.0,29.0,0.0,NaN,0.0,주건축물,314.00,180.380,57.45,627.880,627.880,199.96,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,평스라브,8,0,13.30,5,0,0,0,0,0.0,627.880,0,0.0,0,0.0,7,80.50,1,11.5,20050602,20050603,20050901.0,2005,NaN,NaN,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110125,서울특별시 마포구 성산동 200-260
11,11740-10017967102,2,집합,3,표제부,서울특별시 강동구 강일동 674번지,서울특별시강동구 아리수로93길 40,강일리버파크,11740,11000.0,0.0,674.0,0.0,NaN,NaN,NaN,0,1.174042e+11,11001.0,0.0,40.0,0.0,301동,0.0,주건축물,0.00,564.010,0.00,6401.190,6401.190,0.00,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,공동주택(아파트),10.0,(철근)콘크리트,(철근)콘크리트,78,0,45.60,14,0,3,0,0,0.0,6401.190,0,0.0,0,0.0,0,0.00,0,0.0,20041224,20060926,20090311.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110122,서울특별시 강동구 강일동 674-0


In [120]:
inter2_df.head(2)

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주소
1,11440-2163902,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.30,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.00,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.00,4,46.0,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,서울특별시 마포구 서교동 484-16
6,11590-10018266302,2,집합,3,표제부,서울특별시 동작구 상도동 302-28번지,서울특별시동작구 국사봉8길 64-17,두성쉐르빌,11590,10200.0,0.0,302.0,28.0,NaN,NaN,NaN,1,1.159042e+11,10201.0,0.0,64.0,17.0,NaN,0.0,주건축물,389.48,232.42,59.67,947.74,659.89,169.43,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트,10,0,11.26,4,1,0,0,0,0.0,947.74,0,0.0,0,0.0,8,287.85,2,11.5,20080404,20080405,20080826.0,2008,3190077.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110119,서울특별시 동작구 상도동 302-28


In [124]:
inter2_df['생성년도'] = inter2_df['생성_일자'].apply(lambda x: int(str(x)[:4]))
inter2_df.head(2)

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주소,생성년도
1,11440-2163902,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.30,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.00,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.00,4,46.0,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,서울특별시 마포구 서교동 484-16,2010
6,11590-10018266302,2,집합,3,표제부,서울특별시 동작구 상도동 302-28번지,서울특별시동작구 국사봉8길 64-17,두성쉐르빌,11590,10200.0,0.0,302.0,28.0,NaN,NaN,NaN,1,1.159042e+11,10201.0,0.0,64.0,17.0,NaN,0.0,주건축물,389.48,232.42,59.67,947.74,659.89,169.43,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트,10,0,11.26,4,1,0,0,0,0.0,947.74,0,0.0,0,0.0,8,287.85,2,11.5,20080404,20080405,20080826.0,2008,3190077.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110119,서울특별시 동작구 상도동 302-28,2011


In [125]:
inter2_df['생성년도'].value_counts()

2017    392003
2011    139124
2019     92330
2012     83597
2014     77968
2020     37031
2013     36042
2015     29456
2018     20946
2010     19167
2016     15965
2009     11214
Name: 생성년도, dtype: int64

In [126]:
inter2_df['생성년도'].min()

2009

In [127]:
inter2_df['주소'].nunique()

108313

In [128]:
for addr in inter2_df['주소'].unique():
    addr_df = inter2_df[inter2_df['주소'] == addr]
    if addr_df.shape[0] > 1:
        print(addr_df.shape)
        display(addr_df)
        break

(7, 77)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주소,생성년도
1,11440-2163902,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.3,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,서울특별시 마포구 서교동 484-16,2010
36,11440-21639,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시 마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.3,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,서울특별시 마포구 서교동 484-16,2010
209010,11440-21639,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시 마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.3,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628.0,20030820.00000,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,서울특별시 마포구 서교동 484-16,2010
29153,11440-21639,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시 마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.3,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628.00000,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20170209,서울특별시 마포구 서교동 484-16,2017
56766,11440-21639,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시 마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.3,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628.0,20030820.0,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20170209,서울특별시 마포구 서교동 484-16,2017
31915,11440-21639,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시 마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.3,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628.0,20030820.0,20040323.0,2003.0,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20170209,서울특별시 마포구 서교동 484-16,2017
29334,11440-21639,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시 마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.3,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628.00000,20030820.0,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20170209,서울특별시 마포구 서교동 484-16,2017


In [129]:
inter2_df.drop(columns=['관리_건축물대장_PK'], inplace=True)

In [130]:
inter2_df.shape

(954843, 76)

In [131]:
inter2_df.drop_duplicates(inplace=True)

In [132]:
inter2_df.shape

(854112, 76)

In [133]:
inter2_df.head()

,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주소,생성년도
1,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.300,59.28,393.130,385.210,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.00,4,1,0,0,0,0.0,393.130,0,0.0,0,0.0,0,0.00,4,46.0,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,서울특별시 마포구 서교동 484-16,2010
6,2,집합,3,표제부,서울특별시 동작구 상도동 302-28번지,서울특별시동작구 국사봉8길 64-17,두성쉐르빌,11590,10200.0,0.0,302.0,28.0,NaN,NaN,NaN,1,1.159042e+11,10201.0,0.0,64.0,17.0,NaN,0.0,주건축물,389.48,232.420,59.67,947.740,659.890,169.43,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트,10,0,11.26,4,1,0,0,0,0.0,947.740,0,0.0,0,0.0,8,287.85,2,11.5,20080404,20080405,20080826.0,2008,3190077.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110119,서울특별시 동작구 상도동 302-28,2011
7,2,집합,3,표제부,서울특별시 도봉구 쌍문동 380-24번지,서울특별시도봉구 노해로40길 25,NaN,11320,10500.0,0.0,380.0,24.0,NaN,NaN,NaN,0,1.132041e+11,10501.0,0.0,25.0,0.0,NaN,0.0,주건축물,208.00,81.085,38.98,268.165,268.165,128.93,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트,4,0,10.40,4,0,0,0,0,0.0,268.165,0,0.0,0,0.0,3,0.00,1,0.0,20080318,20080321,20080721.0,2008,3090077.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110202,서울특별시 도봉구 쌍문동 380-24,2011
8,2,집합,3,표제부,서울특별시 마포구 성산동 200-260번지,서울특별시마포구 월드컵북로27길 29,대현하이빌,11440,12500.0,0.0,200.0,260.0,NaN,NaN,NaN,0,1.144041e+11,12501.0,0.0,29.0,0.0,NaN,0.0,주건축물,314.00,180.380,57.45,627.880,627.880,199.96,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,평스라브,8,0,13.30,5,0,0,0,0,0.0,627.880,0,0.0,0,0.0,7,80.50,1,11.5,20050602,20050603,20050901.0,2005,NaN,NaN,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110125,서울특별시 마포구 성산동 200-260,2011
11,2,집합,3,표제부,서울특별시 강동구 강일동 674번지,서울특별시강동구 아리수로93길 40,강일리버파크,11740,11000.0,0.0,674.0,0.0,NaN,NaN,NaN,0,1.174042e+11,11001.0,0.0,40.0,0.0,301동,0.0,주건축물,0.00,564.010,0.00,6401.190,6401.190,0.00,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,공동주택(아파트),10.0,(철근)콘크리트,(철근)콘크리트,78,0,45.60,14,0,3,0,0,0.0,6401.190,0,0.0,0,0.0,0,0.00,0,0.0,20041224,20060926,20090311.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110122,서울특별시 강동구 강일동 674-0,2011


In [135]:
for addr in inter2_df['주소'].unique():
    addr_df = inter2_df[inter2_df['주소'] == addr]
    if addr_df.shape[0] > 1:
        print(addr_df.shape)
        display(addr_df)
        display(addr_df.isna().sum())
        break

(7, 76)


,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,주소,생성년도
1,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.3,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,서울특별시 마포구 서교동 484-16,2010
36,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시 마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.3,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,서울특별시 마포구 서교동 484-16,2010
209010,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시 마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.3,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628.0,20030820.00000,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,서울특별시 마포구 서교동 484-16,2010
29153,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시 마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.3,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628.00000,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20170209,서울특별시 마포구 서교동 484-16,2017
56766,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시 마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.3,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628.0,20030820.0,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20170209,서울특별시 마포구 서교동 484-16,2017
31915,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시 마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.3,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628.0,20030820.0,20040323.0,2003.0,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20170209,서울특별시 마포구 서교동 484-16,2017
29334,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시 마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.3,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628.00000,20030820.0,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20170209,서울특별시 마포구 서교동 484-16,2017


대장_구분_코드          0
대장_구분_코드_명        0
대장_종류_코드          0
대장_종류_코드_명        0
대지_위치             0
도로명_대지_위치         0
건물_명              7
시군구_코드            0
법정동_코드            0
대지_구분_코드          0
번                 0
지                 0
특수지_명             7
블록                7
로트                7
외필지_수             0
새주소_도로_코드         0
새주소_법정동_코드        0
새주소_지상지하_코드       0
새주소_본_번           0
새주소_부_번           0
동_명               7
주_부속_구분_코드        0
주_부속_구분_코드_명      0
대지_면적(㎡)          0
건축_면적(㎡)          0
건폐_율(%)           0
연면적(㎡)            0
용적_률_산정_연면적(㎡)    0
용적_률(%)           0
구조_코드             0
구조_코드_명           0
기타_구조             0
주_용도_코드           0
주_용도_코드_명         0
기타_용도             0
지붕_코드             0
지붕_코드_명           0
기타_지붕             0
세대_수(세대)          0
가구_수(가구)          0
높이(m)             0
지상_층_수            0
지하_층_수            0
승용_승강기_수          0
비상용_승강기_수         0
부속_건축물_수          0
부속_건축물_면적(㎡)      0
총_동_연면적(㎡)        0
옥내_기계식_대수(대)      0


In [136]:
for col in addr_df.columns:
    print(addr_df[col].iloc[0] == addr_df[col].iloc[1])

True
True
True
True
True
False
False
True
True
True
True
True
False
False
False
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
False
True
False
True
True
True
True


In [138]:
np.nan == np.nan

False

In [139]:
inter2_df.fillna('nan', inplace=True)

MemoryError: Unable to allocate 202. MiB for an array with shape (26477472,) and data type object

In [ ]:
inter2_df.drop(columns=['도로명_대지_위치', '새주소_도로_코드', '새주소_법정동_코드'])